In [1]:
import sys; sys.path.append("../")

import pandas as pd

from lib.explore import show_unique
from clean.baton_rouge_so_cprr import clean18

In [2]:
df = clean18()
df

,badge_no,rank_desc,race,sex,birth_year,complainant_type,disposition,action,department_desc,first_name,...,rank_year,rank_month,rank_day,occur_year,occur_month,occur_day,occur_time,data_production_year,agency,uid
0,2206,sergeant,white,male,1984,public,founded,forwarded completed report to administration f...,uniform patrol,william,...,2017,2,6,2018,1,26,21:00,2018,Baton Rouge SO,402619644d9eff018cd39a757bcc5153
1,1711,corporal,white,male,1984,administration,founded,7 day suspension; 6 month loss of take-home ve...,scat,eric,...,2012,2,13,2018,2,20,00:00,2018,Baton Rouge SO,7b22c2ab899789ff98f5cd884b4175be
2,1308,lieutenant,white,male,1968,coworker,founded,resigned,parish prison (training),eddie,...,2006,9,6,2018,3,12,00:00,2018,Baton Rouge SO,1f86bca13a125c7422d192a14159ac21
3,1534,sergeant,black,male,1966,coworker,inconclusive,none,construction services,troy,...,2015,6,15,2018,3,15,00:00,2018,Baton Rouge SO,93c7b5cba8c235cb5aa61b013922d71a
4,2527,deputy,black,female,1986,public,founded,verbal warning from captain flynn about courte...,community policing,pashion,...,2014,12,16,2018,4,8,15:30,2018,Baton Rouge SO,6cf20697312e7736147155175876da95
5,1797,corporal,white,female,1982,supervisor,founded,none,financial crimes,leighann,...,2015,2,9,2018,4,11,10:00,2018,Baton Rouge SO,b91b655b93a3c0de5c46674b3472867f
6,769,captain,black,female,1965,subordinate,unfounded,none,communications,eleanor,...,2014,6,16,2018,4,13,12:00,2018,Baton Rouge SO,d4abd7442439b47d115132746ecdf7da
7,2067,deputy,white,male,1982,public,inconclusive,none,scat,john,...,2008,11,17,2018,4,19,00:00,2018,Baton Rouge SO,b180787c744ce3e7383aa683f40b51ea
8,1519,lieutenant,white,male,1982,administration,founded,5 day suspension,uniform patrol,justin,...,2017,11,27,2018,4,22,00:00,2018,Baton Rouge SO,3f6d99799d87868c8b9f69126f4b331b
9,2363,sergeant,white,male,1983,public,unfounded,none,uniform patrol,dustin,...,2017,11,27,2018,5,10,00:00,2018,Baton Rouge SO,93efca0952d59008ceb2f8f0baaeb7ac


In [5]:
df.columns

Index(['badge_no', 'rank_desc', 'race', 'sex', 'birth_year',
       'complainant_type', 'disposition', 'action', 'department_desc',
       'first_name', 'middle_initial', 'last_name', 'rule_violation',
       'rule_code', 'rank_year', 'rank_month', 'rank_day', 'occur_year',
       'occur_month', 'occur_day', 'occur_time', 'data_production_year',
       'agency', 'uid'],
      dtype='object')